In [1]:
import pandas as pd, numpy as np, sklearn as sklm
pd.set_option('display.max_columns', None) 

import sqlalchemy, sqlite3   
from sklearn.linear_model import LogisticRegression, LinearRegression 

In [2]:
def combine_dfs(dfs):
    master = dfs[0]
    for i in range(1, len(dfs)):
        cols_new = dfs[i].columns.difference(master.columns).tolist() 
        cols_new.append('Player') 
        master = pd.merge(master, dfs[i][cols_new], on="Player") 
    return master 

In [3]:
db_path = "sqlite:///../../DB/ballbase.db" 

season = "2023_24" 

engine = sqlalchemy.create_engine(db_path) 

pergame = pd.read_sql("averages_"+season, con=engine) 
totals = pd.read_sql("totals_"+season, con=engine) 
per100 = pd.read_sql("per100_"+season, con=engine) 
advanced = pd.read_sql("advanced_"+season, con=engine)
playbyplay = pd.read_sql("playbyplay_"+season, con=engine) 

dfs = [pergame, totals, per100, advanced, playbyplay]  

master = combine_dfs(dfs).drop('Tm', axis=1)
master

,Player,Pos,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,ORtg,DRtg,3PAr,AST%,BLK%,BPM,DBPM,DRB%,DWS,FTr,OBPM,ORB%,OWS,PER,STL%,TOV%,TRB%,TS%,USG%,VORP,WS,WS/48
0,Precious Achiuwa,PF-C,24,74,18,1624,7.2,14.4,0.501,0.8,3.0,0.268,6.4,11.4,0.562,2.1,3.4,0.616,5.9,9.1,14.9,3.0,1.4,2.1,2.5,4.4,17.3,114.0,112,0.207,8.4,3.9,-1.4,0.3,20.5,2.2,0.239,-1.7,13.0,1.2,14.6,1.4,13.8,16.7,0.545,15.9,0.2,3.4,0.102
1,Bam Adebayo,C,26,71,71,2416,10.9,21.0,0.521,0.3,0.9,0.357,10.6,20.1,0.528,6.0,8.0,0.755,3.3,11.9,15.2,5.7,1.7,1.4,3.3,3.3,28.2,115.0,109,0.041,18.9,2.7,2.4,1.7,27.3,4.3,0.381,0.8,7.4,2.9,19.8,1.7,12.0,17.3,0.576,24.9,2.7,7.2,0.144
2,Ochai Agbaji,SG,23,78,28,1641,5.2,12.7,0.411,1.8,6.2,0.294,3.4,6.5,0.523,1.1,1.6,0.661,2.2,4.2,6.4,2.4,1.4,1.3,1.9,3.4,13.4,102.0,121,0.487,6.6,2.4,-4.4,-0.9,9.6,0.6,0.129,-3.5,4.9,-0.5,7.7,1.4,12.3,7.2,0.497,13.4,-1.0,0.1,0.002
3,Santi Aldama,PF,23,61,35,1618,7.5,17.2,0.435,3.2,9.2,0.349,4.3,8.0,0.534,1.6,2.6,0.621,2.2,8.5,10.6,4.2,1.3,1.6,2.1,2.7,19.8,109.0,114,0.535,13.3,3.1,-0.2,0.4,19.7,2.0,0.153,-0.7,4.7,0.4,13.1,1.3,10.2,11.9,0.539,17.9,0.7,2.4,0.070
4,Nickeil Alexander-Walker,SG,25,82,20,1921,6.1,13.8,0.439,3.4,8.6,0.391,2.7,5.2,0.517,1.3,1.7,0.800,0.9,3.4,4.3,5.2,1.6,1.1,2.0,3.7,16.9,115.0,111,0.623,14.5,1.9,0.2,1.3,7.5,2.8,0.121,-1.1,2.2,1.5,10.9,1.6,11.8,4.9,0.578,14.8,1.1,4.3,0.107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,Thaddeus Young,PF,35,33,6,439,7.2,11.9,0.602,0.1,0.8,0.143,7.1,11.1,0.634,0.7,1.7,0.400,5.1,6.3,11.4,6.3,2.5,0.6,1.7,5.4,15.1,130.0,116,0.065,17.9,1.0,0.3,0.8,14.5,0.4,0.139,-0.5,11.7,0.9,15.9,2.5,11.6,13.0,0.598,12.6,0.2,1.3,0.138
568,Trae Young,PG,25,54,54,1942,10.7,24.9,0.430,4.3,11.6,0.373,6.4,13.3,0.479,8.6,10.1,0.855,0.6,3.1,3.7,14.4,1.8,0.3,5.8,2.7,34.3,117.0,122,0.465,45.1,0.5,2.6,-2.3,7.3,0.6,0.404,4.9,1.2,4.0,20.3,1.8,16.5,4.2,0.585,30.5,2.2,4.6,0.114
569,Omer Yurtseven,C,25,48,12,545,8.8,16.3,0.538,0.4,2.1,0.208,8.3,14.2,0.588,1.7,2.5,0.679,6.4,12.0,18.4,2.6,0.7,1.6,3.3,4.6,19.7,112.0,118,0.130,7.7,2.9,-3.0,-1.5,27.3,0.4,0.152,-1.6,14.7,0.3,15.0,0.7,15.9,21.0,0.565,17.9,-0.1,0.7,0.062
570,Cody Zeller,C,31,43,0,320,4.0,9.5,0.419,0.2,0.5,0.333,3.8,9.0,0.424,3.5,5.8,0.605,7.4,9.8,17.2,6.0,1.4,0.8,2.5,6.9,11.6,119.0,112,0.048,15.2,1.5,-2.6,0.3,22.0,0.4,0.613,-2.9,16.9,0.4,12.8,1.4,16.9,19.5,0.483,12.9,0.0,0.8,0.124


## Export to Database 

In [9]:
engine = sqlalchemy.create_engine("sqlite:///../../DB/ballbase.db", echo=False) 
master.to_sql("master_"+season, con=engine, if_exists="replace", index=False)  

572

In [10]:
# Create HTML table version 
master = master.to_html(table_id="table", classes = "table table-hover table-stripped table-bordered", index=False)

# Create page content 
master = f"""
{{%extends 'TableBase.html'%}}

{{% block table %}}
{master}
{{% endblock %}}
"""

In [11]:
def write_to_file(path, content): 
    file = open(path, "w", encoding="utf-8") 
    file.write(content) 
    file.close 

path = f"../../templates/Tables/Master.html" 
write_to_file(path, master) 
    
    